## **Setup**

In [1]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: ok


In [1]:
import os
import sys
import os.path as op
import numpy as np
from functools import partial
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks import ModelCheckpoint, StochasticWeightAveraging
from pytorch_lightning.loggers import WandbLogger

import torch
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

sys.path.append("..")
from mtecg import MultiTaskDataset, MultiTaskModel
from mtecg.utils import load_ecg_dataframe


SEED = 42
np.random.seed(SEED)
seed_everything(SEED, workers=True)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.cuda.manual_seed(SEED)

c:\Anaconda3\envs\ecg\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda3\envs\ecg\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: llow is an invalid version and will not be supported in a future release
  warnings.warn(
Global seed set to 42


In [2]:
lvef_threshold = 50
image_size= (384, 384)
batch_size = 16
num_epochs = 10

configs = {
    "in_channels": 3,
    "learning_rate": 5e-4,
    "use_timm": True,
    "pretrained": True,
    "backbone": "resnet34d",
    "latent_dim": 512,
    "scar_class": 2,
    "lvef_class": 2,
    "scar_lvef_loss_ratio": [0.7, 0.3],
    "bias_head": True,
    # Specify the device.
    "device": "cuda",
    }

## **Prepare the data**

In [3]:
image_dir = "../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_images_new/"
csv_path = "../../ECG_EF_Clin_train_dev_new.csv"

df = load_ecg_dataframe(csv_path, image_dir)
print(f"Number of images: {len(df)}")
print(f"Unique splits: {df['split'].unique()}")
df.head(5)

Number of images: 13343
Unique splits: ['old_train' 'old_valid' 'old_test' 'new_train' 'new_valid']


,run_num,train_80_percent,develop_10_percent,file_name,lvef,scar_cad,hcm,mri_date,month,year,...,dm,ht,mi,pci,cabg,ua,chest pain,dyspnea,path,split
0,1,1.0,NaN,2009_420521391,59.9,0,0,2552-08-01 00:00:00,8,2009,...,0,1,0,0,0,0,1,0,../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_i...,old_train
1,2,1.0,NaN,2009_472422791,81.7,0,0,2552-08-01 00:00:00,8,2009,...,0,1,0,0,0,0,1,0,../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_i...,old_train
2,3,1.0,NaN,2009_451191451,64.7,0,0,2552-08-01 00:00:00,8,2009,...,0,1,0,0,0,0,1,1,../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_i...,old_train
3,4,1.0,NaN,2009_512029431,10.7,1,0,2552-08-01 00:00:00,8,2009,...,1,0,1,1,0,0,0,1,../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_i...,old_train
4,5,1.0,NaN,2009_461543281,19.3,1,0,2552-08-04 00:00:00,8,2009,...,0,1,0,0,0,0,1,1,../../ecg/ecg-cnn-local/siriraj_data/ECG_MRI_i...,old_train


In [4]:
pretrain_train_df = df[df.split.isin(["old_train"])].reset_index()
pretrain_valid_df = df[df.split.isin(["old_valid"])].reset_index()

transfer_train_df = df[df.split.isin(["new_train"])].reset_index()
transfer_valid_df = df[df.split.isin(["new_valid"])].reset_index()

print(f"Number of pretrain train images: {len(pretrain_train_df)}")
print(f"Number of pretrain valid images: {len(pretrain_valid_df)}")
print("="*50)
print(f"Number of transfer train images: {len(transfer_train_df)}")
print(f"Number of transfer valid images: {len(transfer_valid_df)}")

Number of pretrain train images: 5795
Number of pretrain valid images: 1419
Number of finetune train images: 3598
Number of finetune valid images: 1486


In [6]:
train_transform = A.Compose([
    A.Resize(*image_size),
    A.Blur(blur_limit=3, p=0.2),
    A.RandomBrightnessContrast(),
    A.MotionBlur(p=0.2),
    A.Normalize(),
    ToTensorV2(),
])

valid_transform = A.Compose([
    A.Resize(*image_size),
    A.Normalize(),
    ToTensorV2()
])

In [7]:
pretrain_train_ds = MultiTaskDataset(pretrain_train_df, train_transform, lvef_threshold=lvef_threshold)
pretrain_valid_ds = MultiTaskDataset(pretrain_valid_df, valid_transform, lvef_threshold=lvef_threshold)
pretrain_train_loader = DataLoader(pretrain_train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
pretrain_valid_loader = DataLoader(pretrain_valid_ds, batch_size=batch_size, shuffle=False, pin_memory=True)


transfer_train_ds = MultiTaskDataset(transfer_train_df, train_transform, lvef_threshold=lvef_threshold)
transfer_valid_ds = MultiTaskDataset(transfer_valid_df, valid_transform, lvef_threshold=lvef_threshold)
transfer_train_loader = DataLoader(transfer_train_ds, batch_size=batch_size, shuffle=True, pin_memory=True)
transfer_valid_loader = DataLoader(transfer_valid_ds, batch_size=batch_size, shuffle=False, pin_memory=True)

## **Pretrain (on Old-format Data)**

In [ ]:
parent_pretrained_save_dir = f"../trained_models/multi_task_old_format"
os.makedirs(parent_pretrained_save_dir, exist_ok=True)

run_suffix = f"{image_size[0]}_LVEF{str(lvef_threshold)}"
run_name = f"{configs['backbone']}_{run_suffix}"

In [8]:
model = MultiTaskModel(**configs)

In [9]:
import wandb
project_name = f"ecg-multi-task-transfer-stage1"

os.environ["WANDB_NOTEBOOK_NAME"] = "ecg-multi-task-transfer.ipynb"
run = wandb.init(project = project_name, save_code = True)
run.log_code(".", include_fn = lambda path: path.endswith(".py") or path.endswith(".ipynb"))
run.config.update({"batch_size": batch_size,})

checkpoint_callback = ModelCheckpoint(
    filename = configs["backbone"] + "{val_acc:.2f}",
    save_top_k = 1,
    verbose = True,
    monitor = "val_loss",
    mode = "min",
)

logger = WandbLogger(
    project = project_name,
    name = configs["backbone"],
    # log_model = "all", # set to True to log at the end
)

logger.watch(
    model, 
    # log_freq=300, # uncomment to log gradients
    log_graph = True,
)

wandb: Currently logged in as: name_atirut. Use `wandb login --relogin` to force relogin


c:\Anaconda3\envs\ecg\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: llow is an invalid version and will not be supported in a future release
  warnings.warn(
C:\Users\namea\AppData\Roaming\Python\Python38\site-packages\pytorch_lightning\loggers\wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


In [ ]:
trainer = Trainer(
    accelerator="gpu",
    logger = logger,
    max_epochs = num_epochs,
    callbacks = [checkpoint_callback, StochasticWeightAveraging(1e-3)],
)

trainer.fit(
    model,
    train_dataloaders = pretrain_train_loader,
    val_dataloaders = pretrain_valid_loader,
)

In [ ]:
trainer.save_checkpoint(op.join(parent_pretrained_save_dir, run_name, "model.ckpt"))
model.save_configs(op.join(parent_pretrained_save_dir, run_name))

A.save(train_transform, op.join(parent_pretrained_save_dir, run_name, "train_transform.json"))
A.save(valid_transform, op.join(parent_pretrained_save_dir, run_name, "transform.json"))

## **Finetune (on New-format Data)**

In [ ]:
parent_transferred_save_dir = f"../trained_models/multi_task_transferred"
os.makedirs(parent_transferred_save_dir, exist_ok=True)

run_suffix = f"{image_size[0]}_LVEF{str(lvef_threshold)}"
run_name = f"{configs['backbone']}_{run_suffix}"

In [ ]:
pretrained_model_path = op.join(parent_pretrained_save_dir, run_name)
# Explicitly specify train=True to instantiate the model with loss functions on the correct device.
model = MultiTaskModel.from_configs(pretrained_model_path, train=True, device="cuda")
configs = model.configs

In [ ]:
import wandb
project_name = f"ecg-multi-task-transfer-stage2"

os.environ["WANDB_NOTEBOOK_NAME"] = "ecg-multi-task-transfer.ipynb"
run = wandb.init(project = project_name, save_code = True)
run.log_code(".", include_fn = lambda path: path.endswith(".py") or path.endswith(".ipynb"))
run.config.update({"batch_size": batch_size,})

checkpoint_callback = ModelCheckpoint(
    filename = configs["backbone"] + "{val_acc:.2f}",
    save_top_k = 1,
    verbose = True,
    monitor = "val_loss",
    mode = "min",
)

logger = WandbLogger(
    project = project_name,
    name = configs["backbone"],
    # log_model = "all", # set to True to log at the end
)

logger.watch(
    model, 
    # log_freq=300, # uncomment to log gradients
    log_graph = True,
)

In [ ]:
trainer = Trainer(
    accelerator="gpu",
    logger = logger,
    max_epochs = num_epochs,
    callbacks = [checkpoint_callback, StochasticWeightAveraging(1e-3)],
)

trainer.fit(
    model,
    train_dataloaders = transfer_train_loader,
    val_dataloaders = transfer_valid_loader,
)

In [ ]:
trainer.save_checkpoint(op.join(parent_transferred_save_dir, run_name, "model.ckpt"))
model.save_configs(op.join(parent_transferred_save_dir, run_name))

A.save(train_transform, op.join(parent_transferred_save_dir, run_name, "train_transform.json"))
A.save(valid_transform, op.join(parent_transferred_save_dir, run_name, "transform.json"))